# Neighbs

In [1]:
from koselleck import *

In [2]:
def get_pathdf_models(period_len=5):
    pathdf=get_model_paths_df(PATH_MODELS_BPO, 'model.bin').sort_values(['period_start','run'])
    pathdf['period']=[f'{x}-{y}' for x,y in zip(pathdf.period_start, pathdf.period_end)]
    pathdf['period_len']=pathdf.period_end - pathdf.period_start
    if period_len: pathdf=pathdf[pathdf.period_len==period_len]
    return pathdf[~pathdf.period.isnull()]


In [3]:
dfmodels = get_pathdf_models().query(
    'period_len==5 & run<="run_10" & period_start>=1720'
)
dfmodels

,corpus,period_start,period_end,path,path_vocab,run,period,period_len
525,bpo,1720,1725,/home/ryan/github/koselleck/data/models/bpo/17...,/home/ryan/github/koselleck/data/models/bpo/17...,run_01,1720-1725,5
532,bpo,1720,1725,/home/ryan/github/koselleck/data/models/bpo/17...,/home/ryan/github/koselleck/data/models/bpo/17...,run_02,1720-1725,5
514,bpo,1720,1725,/home/ryan/github/koselleck/data/models/bpo/17...,/home/ryan/github/koselleck/data/models/bpo/17...,run_03,1720-1725,5
523,bpo,1720,1725,/home/ryan/github/koselleck/data/models/bpo/17...,/home/ryan/github/koselleck/data/models/bpo/17...,run_04,1720-1725,5
527,bpo,1720,1725,/home/ryan/github/koselleck/data/models/bpo/17...,/home/ryan/github/koselleck/data/models/bpo/17...,run_05,1720-1725,5
...,...,...,...,...,...,...,...,...
650,bpo,1965,1970,/home/ryan/github/koselleck/data/models/bpo/19...,/home/ryan/github/koselleck/data/models/bpo/19...,run_06,1965-1970,5
651,bpo,1965,1970,/home/ryan/github/koselleck/data/models/bpo/19...,/home/ryan/github/koselleck/data/models/bpo/19...,run_07,1965-1970,5
657,bpo,1965,1970,/home/ryan/github/koselleck/data/models/bpo/19...,/home/ryan/github/koselleck/data/models/bpo/19...,run_08,1965-1970,5
648,bpo,1965,1970,/home/ryan/github/koselleck/data/models/bpo/19...,/home/ryan/github/koselleck/data/models/bpo/19...,run_09,1965-1970,5


In [23]:
def do_get_neighbs(dfpath,words=None,internal_to_words=False,k=50):
    model_path=dfpath.iloc[0].path
    if not words: words=get_valid_words()
    m=load_model(model_path)
    words=set(words)&set(m.wv.key_to_index.keys())
    odf pd.DataFrame([
        {
            'word':w,
            'neighb':', '.join([
                x
                for x,y in m.wv.most_similar(w,topn=k * (5 if internal_to_words else 1))
                if not internal_to_words or (x in words)
            ][:k])
        }
        for w in words
    ])

In [25]:
# do_get_neighbs(dfmodels.iloc[5:10]).query('word=="value"')

In [26]:
def do_combine_neighbs(dfgrp,k=25):
    counts=Counter()
    ranks=defaultdict(list)
    for nstr in dfgrp.neighb:
        for i,w in enumerate(nstr.split(', ')[:k]):
            ranks[w]+=[i]
            counts[w]+=1
    for w in ranks: ranks[w]=np.mean(ranks[w])
    countranks=dict((w, (1-(ranks[w]/len(ranks)))+counts[w]) for w in set(ranks.keys())&set(counts.keys()))
    ostr=', '.join(
        f'{w} ({counts[w]}|{round(ranks[w])+1})'
        for w in list(sorted(
            countranks.keys(),
            key=lambda _w: -countranks[_w]
        ))[:k]
    )
    return pd.DataFrame([{'neighb':ostr}])

def combine_neighbs(dfneighbs,lim=None,k=25,num_proc=1,**y):
    return pmap_groups(
        do_combine_neighbs,
        dfneighbs.iloc[:lim],groupby(['word','period']),
        kwargs=dict((k=k)),
        num_proc=num_proc,
        **y
    )

In [28]:
FN_ALL_NEIGHBS=os.path.join(PATH_DATA,'data.all_local_neighbs.v2.pkl')

def get_all_neighbors(fnfn=FN_ALL_NEIGHBS, k=25, lim=None, num_proc=1):
    if os.path.exists(fnfn): return read_df(FN_ALL_NEIGHBS)
    # otherwise gen
    odf=pmap_groups(
        do_get_neighbs,
        dfmodels.iloc[:lim].groupby(['corpus','period','run']),
        num_proc=4
    )
    odf_combined = combine_neighbs(odf, k=k, lim=lim, num_proc=num_proc)
    odf.to_pickle(FN_ALL_NEIGHBS)
    return odf

In [ ]:
dfneighbs = get_all_neighbors(lim=100)
dfneighbs

Mapping do_get_neighbs [x4]:  64%|██████▍   | 64/100 [04:01<02:46,  4.63s/it]

In [8]:
dfneighbs_f=dfneighbs.query('run<="run_02"')
# dfneighbs_f

In [10]:
dfn=combine_neighbs(dfneighbs_f)

Mapping do_combine_neighbs [x7]: 100%|██████████| 266698/266698 [03:38<00:00, 1221.06it/s]


In [11]:
dfn

neighb
word        period                                                      
abandonment 1800-1805  pedantry (1|1), amelioration (1|2), mechanism ...
            1805-1810  restitution (1|1), revue (1|1), dismissal (1|2...
            1810-1815  armament (2|5), acquiescence (2|11), stability...
            1815-1820  development (2|3), instructor (1|1), enforceme...
            1820-1825  individuality (2|3), manifestation (1|1), embr...
...                                                                  ...
zoology     1920-1925  obituary (1|1), tanner (1|2), commemoration (1...
            1925-1930  professor (1|1), biography (1|2), contribution...
            1940-1945  tutor (1|1), cam (1|2), county (1|3), chemistr...
            1950-1955  preacher (1|1), evangelist (1|2), benefactor (...
            1960-1965  organist (2|11), billiard (1|1), biology (1|1)...

[266698 rows x 1 columns]

In [17]:
pd.options.display.max_colwidth=None
dfn

neighb
word        period                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
abandonment 1800-1805                                pedantry (1|1), amelioration (1|2), mechanism (1|3), intensity (1|4), ague (1|5), inquisition (1|6), tableau (1|7), puzzle (1|8), texture (1|9), twig (1|10), biscuit (1|11), subtlety (1|12), publicity (1|13), approval (1|14), competence (1|15), stoppage (1|16), drug (1|17), abstraction (1|18), acme (1|19), substitution (1|20), infatuation (1|21), abstinence (1|22), physique (1|23), offender (1|24), remuneration (1|25)
            1805-1810                              restitution (1|1), revue (1|1), dismissal (1|2), tableau (1|2), registry (1|3), discontent (1|3), bane (1|4), seizure (1|4), adhesion (1|5), distribution (1|5), magnetism (1|6), negotiation (1|6), violation (1|7), impostor (1|7), chef (1|8), inquisition (1|8), litigation (1|9), transfer (1|9), pursuance (1|10), fortification (1|10), confederacy (1|11), angler (1|11), stratagem (1|12), thinker (1|12), condensation (1|13)
            1810-1815     armament (2|5), acquiescence (2|11), stability (2|17), monarchy (2|18), project (2|19), mediation (2|23), subjection (1|1), evacuation (1|1), adherence (1|2), administration (1|3), defending (1|3), insurrection (1|4), fulfilment (1|4), defeat (1|5), usurpation (1|5), democracy (1|6), executive (1|6), bloodshed (1|7), liberation (1|7), desperation (1|8), furtherance (1|8), guardianship (1|9), government (1|10), negotiation (1|11), renewal (1|11)
            1815-1820          development (2|3), instructor (1|1), enforcement (1|1), condescension (1|2), admirer (1|3), egotism (1|3), enlargement (1|4), impropriety (1|5), thinker (1|5), concentration (1|6), imperfection (1|6), predominance (1|7), excitement (1|7), detestation (1|8), serenity (1|8), revolt (1|9), faculty (1|9), adviser (1|10), malignity (1|10), partiality (1|11), evacuation (1|11), incentive (1|12), magnanimity (1|12), sloth (1|13), vibration (1|13)
            1820-1825  individuality (2|3), manifestation (1|1), embryo (1|2), renunciation (1|2), permanence (1|3), intrepidity (1|3), arsenal (1|4), violation (1|4), derangement (1|5), epitome (1|6), preponderance (1|6), emulation (1|7), toleration (1|7), retrospect (1|8), disregard (1|8), predominance (1|9), annihilation (1|9), inexperience (1|10), illumination (1|10), heroism (1|11), criterion (1|11), probity (1|12), avowal (1|12), prevention (1|13), advocacy (1|13)
...                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    ...
zoology     1920-1925                                                                                                                                                                                                                                                                                                 obituary (1|1), tanner (1|2), commemoration (1|3), warden (1|4), basil (1|5), warren (1|6), booth (1|7), choirmaster (1|8), deputy (1|9), croker (1|10), buck (1|11)
            1925-1930              professor (1|1), biography (1|2), contribution (1|3), handbook (1|4), appendix (1|5), philology (1|6), introduction (1|7), dictionary (1|8), geology (1|9), aq

In [18]:
dfn.loc['circulation']

,neighb
period,
1720-1725,"debt (2|9), efficacy (2|9), minute (2|10), trash (2|13), absence (2|17), tincture (2|20), blood (1|1), currency (1|1), filth (1|2), stamp (1|2), respect (1|3), rubbish (1|3), loon (1|4), proportion (1|5), debility (1|5), tenderness (1|7), patient (1|8), remedy (1|8), animal (1|9), stomach (1|10), debtor (1|11), dissolution (1|11), revenue (1|12), pressure (1|13), deference (1|13)"
1725-1730,"thickness (2|1), compression (2|3), fluid (2|4), sulphur (2|6), animal (2|8), climate (2|13), moisture (2|14), brain (2|18), culture (2|23), vegetable (1|2), glass (1|4), juice (1|5), thermometer (1|5), operation (1|6), smell (1|7), evaporation (1|8), consumption (1|10), atmosphere (1|10), gout (1|11), evacuation (1|11), stool (1|12), melting (1|12), sphere (1|13), perspiration (1|13), vibration (1|15)"
1730-1735,"premium (2|1), annuity (2|3), boil (2|3), dit (2|5), grain (2|7), oats (2|9), loaf (2|18), malt (2|19), cop (2|19), barley (2|20), sos (1|4), pillar (1|5), cream (1|6), pastry (1|6), cookery (1|7), opening (1|8), bank (1|9), pith (1|9), sip (1|10), assurance (1|11), beech (1|11), meal (1|12), sherry (1|13), transfer (1|13), halt (1|14)"
1735-1740,"premium (2|1), salt (2|2), copper (2|7), loaf (2|17), soot (1|3), appetite (1|3), sos (1|4), oats (1|4), juice (1|5), flank (1|6), digestion (1|6), sop (1|7), animal (1|7), dit (1|8), indigestion (1|8), otter (1|9), malt (1|9), rye (1|10), loin (1|11), fibre (1|11), loan (1|12), plunge (1|12), blossom (1|13), circus (1|14), cop (1|14)"
1740-1745,"premium (2|1), salt (2|2), assurance (2|3), annuity (2|6), oats (2|8), loan (2|11), bait (2|14), flu (1|4), loin (1|5), sooth (1|5), sos (1|6), tick (1|7), barley (1|7), bank (1|8), cramp (1|9), bond (1|10), pet (1|11), rye (1|12), gum (1|12), wheat (1|13), fruit (1|14), stock (1|14), stomach (1|15), malt (1|15), sou (1|16)"
1745-1750,"fluid (2|1), digestion (2|3), elasticity (2|5), vegetable (2|6), ferment (2|7), nerve (2|7), vapour (2|11), animal (2|11), juice (2|12), brain (2|13), heat (2|13), mineral (2|13), stomach (2|15), appetite (2|15), respiration (1|6), fruit (1|9), moisture (1|10), velocity (1|11), perspiration (1|11), decay (1|13), indigestion (1|15), soil (1|15), pressure (1|16), tension (1|17), growth (1|19)"
1750-1755,"acid (2|2), fluid (2|3), growth (2|4), nerve (2|9), juice (2|9), tartar (2|10), digestion (2|11), mineral (2|12), perspiration (2|12), texture (2|13), elasticity (2|14), salt (2|15), pressure (2|16), blood (2|20), velocity (1|3), vinegar (1|5), fermentation (1|5), moisture (1|6), nourishment (1|9), bladder (1|10), pus (1|13), solidity (1|14), ferment (1|15), metal (1|16), evaporation (1|17)"
1755-1760,"pressure (2|3), fluid (2|5), nourishment (2|5), perspiration (2|10), evaporation (2|11), vapour (2|11), texture (2|17), gravity (2|19), juice (2|21), vegetation (1|2), atmosphere (1|4), mercury (1|5), acid (1|5), diminution (1|6), digestion (1|7), agriculture (1|8), scale (1|9), expansion (1|9), multiplication (1|10), fermentation (1|10), deficiency (1|11), absorption (1|11), vegetable (1|12), elasticity (1|13), mixing (1|13)"
1760-1765,"acid (2|5), pressure (2|5), atmosphere (2|6), perspiration (2|7), fermentation (2|8), fluid (2|9), stomach (2|9), alkali (2|11), mixture (2|11), membrane (2|14), depth (2|16), inflammation (2|16), velocity (2|17), cavity (2|21), surface (2|22), growth (1|2), texture (1|3), tube (1|4), hardness (1|6), irritation (1|8), grain (1|9), product (1|9), flavour (1|13), sulphur (1|15), gravity (1|16)"
